This Jupyter NB has all of the data preprocessing and modeling for the Team of the Season Predictor. In the Notebook, I will go through data preprocessing, various ML arcitectures that might work for this Classification problem, stastical comparisons between the models, and a final conclusion on what model I will use and why I chose it. Some more insight into the data can be found in WebScrape Juptyer NB found in the Practice folder.

# Data Preprocessing

The first part of this code is from `Webscrape.ipynb`in the Practice folder. 

In [1]:
"""IMPORTS"""
# Data Preprocessing
import pandas as pd


# Web Scraping
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re

In [3]:
""" WEBSCRAPING PLAYER STATS """

# Player Stats from FBREF
url_2023_2024 = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'
url_2022_2023 = 'https://fbref.com/en/comps/9/2022-2023/stats/2022-2023-Premier-League-Stats'
url_2021_2022 = 'https://fbref.com/en/comps/9/2021-2022/stats/2021-2022-Premier-League-Stats'
url_2020_2021 = 'https://fbref.com/en/comps/9/2020-2021/stats/2020-2021-Premier-League-Stats'
url_2019_2020 = 'https://fbref.com/en/comps/9/2019-2020/stats/2019-2020-Premier-League-Stats'
url_2018_2019 = 'https://fbref.com/en/comps/9/2018-2019/stats/2018-2019-Premier-League-Stats'
url_2017_2018 = 'https://fbref.com/en/comps/9/2017-2018/stats/2017-2018-Premier-League-Stats'

urls = [ url_2023_2024 , url_2022_2023 , url_2021_2022 , url_2020_2021 , url_2019_2020, url_2018_2019, url_2017_2018]
player_stat_dfs = []


for link in urls:
    print(link)
    temp = pd.read_html(
        requests.get(link).text.replace('<!--','').replace('-->','')
        ,attrs={'id':'stats_standard'}
    )[0]
    
    # temp = pd.read_html(
    #     requests.get(link).text ,  attrs={'id':'stats_standard'}
    # )[0]
    
    temp_data = pd.DataFrame(temp)
    
    player_stat_dfs.append(temp_data)
    


"""PULLING TEAM OF THE SEASON INFO"""
"""TARGET SEASONS: 2017-2018 - 2023-2024"""


# Get URLS for TOTS info
url_tots_wiki_2020s = 'https://en.wikipedia.org/wiki/PFA_Team_of_the_Year_(2020s)'
url_tots_wiki_2010s = 'https://en.wikipedia.org/wiki/PFA_Team_of_the_Year_(2010s)'

tots_urls = [url_tots_wiki_2020s , url_tots_wiki_2010s]
tots_dfs = []

for url in tots_urls:
    
    # Send an HTTP GET request to the URL
    temp_response = requests.get(url)
    
    if temp_response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(temp_response.content, 'html.parser')
        
        # Find Where all the PL Headers are 
        h4_headers = soup.find_all('h4')
        
        # Get PL Tags
        premier_league_tags = [tag for tag in h4_headers if 'Premier League' in tag.text]
        
        
        
        for header in premier_league_tags:
            # Find the next sibling table tag
            table = header.find_next_sibling('table')
            
            # If a table is found, print it
            if table:
                df = pd.read_html(StringIO(str(table)))[0]
                tots_dfs.append(df) 
    
    else:
        print("Failed to retrieve the page. Status code:", temp_response.status_code)

https://fbref.com/en/comps/9/stats/Premier-League-Stats


/var/folders/_x/czhgmpkn0lx4yd57n3w9bwv00000gn/T/ipykernel_84267/2674133702.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp = pd.read_html(


ValueError: No tables found

In [ ]:
"""PUT TABLES INTO DICTIONARY BY YEAR"""

yr_playerStat_dict = {
    '2023-2024' : player_stat_dfs[0],
    '2022-2023' : player_stat_dfs[1],
    '2021-2022' : player_stat_dfs[2],
    '2020-2021' : player_stat_dfs[3],
    '2019-2020' : player_stat_dfs[4],
    '2018-2019' : player_stat_dfs[5],
    '2017-2018' : player_stat_dfs[6]
}

"""MAPPING TABLES TO APROPRIATE YEAR IN DICTIONARY (year -> TOTS Table)"""

yr_totsNoms_dict = {
    '2019-2020' : tots_dfs[0] ,
    '2020-2021' : tots_dfs[1] ,
    '2021-2022' : tots_dfs[2] ,
    '2022-2023' : tots_dfs[3] , 
    '2017-2018' : tots_dfs[-2] , 
    '2018-2019' : tots_dfs[-1] , 
}

In [ ]:
list_seasons = ['2017-2018' , '2018-2019' , '2019-2020' , '2020-2021', '2021-2022' , '2022-2023'] # Exclude current Season

# All DataFrames Stored
merged_dataframes = []

for szn in list_seasons:
    tmp_tots_stats = yr_totsNoms_dict[szn]
    tmp_plyr_stats = yr_playerStat_dict[szn]
    
    # Strip Player name to get rid of special chars
    tmp_tots_stats['Player'] = tmp_tots_stats['Player'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x).strip())
    
    # Updating Names for TOTS Nominees Table to match Player Stats DF
    tmp_tots_stats = tmp_tots_stats.rename(columns={'App.': 'Tots_Apps' , 'Pos.': 'Pos' , 'Club': 'Squad'})
    
    # Drop Upper Level Layers
    tmp_plyr_stats.columns = tmp_plyr_stats.columns.droplevel([0])
    
    # Merge the DataFrames
    # Join tables, keep all of info from player stats , join on Player name and squad
    tmp_df_merged = pd.merge(tmp_plyr_stats, tmp_tots_stats, on=['Player', 'Squad'], how='left')
    
    #Fix Pos_x 
    tmp_df_merged['Pos'] = tmp_df_merged['Pos_x'].str.split(',').str[0]
    
    # If Tots_Apps = NaN
    # Make that = 0
    tmp_df_merged['Tots_Apps'] = tmp_df_merged['Tots_Apps'].fillna(0)
    
    # Nation 
    # Use the ALL CAPS Countries
    tmp_df_merged['Nation'] = tmp_df_merged['Nation'].str.split().str[-1]
    
    # Drop Matches, Rk , Extra Pos's
    tmp_df_merged = tmp_df_merged.drop(columns=['Rk', 'Matches' , 'Pos_x' , 'Pos_y' ])
    
    merged_dataframes.append(tmp_df_merged)
    
    
for mdf in merged_dataframes: mdf.drop(mdf[mdf['Player'] == 'Player'].index, inplace=True)
    



In [ ]:
print(len(merged_dataframes))
merged_dataframes[0]

#### Cleaning the Data

In [ ]:
# Add in the target column
for mdf in merged_dataframes:
    mdf['TOTS'] = mdf['Tots_Apps'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
merged_dataframes[4].head(20)

#### Decision Making

In [ ]:
total_plyrs = 0
tots_plyrs = 11 * len(merged_dataframes)
for merged_df in merged_dataframes:
    total_plyrs += len(merged_df)

print(f"Players in TOTS over 6 years: { tots_plyrs }")
print(f"Players with stats over 6 years: { total_plyrs }")
print(f"Percentage of players that are in TOTS: % {tots_plyrs /total_plyrs * 100 }")

From the code above, we can see that we have major class imbalance when putting the DataFrames together. We will see how the model works out and if it performs well when the data is put together across different seasons. 

The reasoning behind putting the data together is that I am hoping to find a consistent standard on certain stats that lead to a player being nominated for TOTS. If I can establish this relationship, then I can use the stats for a players current season, find out what they average to, find an expected total, and use that to predict if they will make TOTS or not.

For this predict method we will have to look into Fbeta scores and see how our model performs using a confusion matrix. I also need to establish if False Positives or False Positives are more important.

In [ ]:
"""JOINING THE TABLES"""
df = pd.concat(merged_dataframes)
df

In this section, we are going to make the 'Squad' column into a ordinal categorical variable by ordering the squads by average league position over the time we are getting the data. We are doing this because when looking at TOTS data, it is evident that teams that place higher in the standings tend to have more players in the TOTS. This is a big assumption I am making, so if it does not seem to have an affect on what we get from the model, I will find another way to handle this.

To accomplish this, we need to do more webscraping from fbref to find the tables and get the average position for each team that played in the Premier League in the timeframe from the 2017-18 season to the 2022-2023 season. 

In [ ]:
# list_seasons = ['2017-2018' , '2018-2019' , '2019-2020' , '2020-2021', '2021-2022' , '2022-2023'] # Exclude current Season

#     # url_temp = f'https://fbref.com/en/comps/9/{szn}/stats/{szn}-Premier-League-Stats'
# url_temp = f'https://fbref.com/en/comps/9/2017-2018/stats/2017-2018-Premier-League-Stats'

# # https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats

# for szn in list_seasons:
#     url_temp = f'https://fbref.com/en/comps/9/{szn}/{szn}-Premier-League-Stats'
#     print(url_temp)
#     # Send an HTTP GET request to the URL
#     temp_response = requests.get(url_temp)
    
#     if temp_response.status_code == 200:
#         # Parse the HTML content
#         soup = BeautifulSoup(temp_response.content, 'html.parser')
        
#         # Find Where all the Regular Season Headers are 
#         h2_headers = soup.find_all('h2')
        
#         # Get PL Tags
#         standings_tags = [tag for tag in h2_headers if 'Regular season Table' in tag.text]
#         print(standings_tags)
        
#         for header in premier_league_tags:
#             # Find the next sibling table tag
#             table = header.find_next_sibling('table')
            
#             # If a table is found, print it
#             if table:
#                 df = pd.read_html(StringIO(str(table)))[0]
#                 tots_dfs.append(df) 
    
#     else:
#         print("Failed to retrieve the page. Status code:", temp_response.status_code)
